In [38]:
import numpy as np
import pandas as pd
from pyspark.sql import SparkSession
from pyspark import SparkContext

import os
os.environ['JAVA_HOME'] = 'C:\Program Files\Java\jdk-18.0.2'
os.environ['SPARK_HOME'] = 'C:\Program Files\spark-3.3.0-bin-hadoop3'

<h2>Extract</h2>

In [39]:
#Create a spark session

spark = SparkSession.builder.appName('Read JSON File into DataFrame').getOrCreate()
print(spark.sparkContext.uiWebUrl)
#Read JSON and assign to pyspark dataframes

customer_df = spark.read.json('cdw_sapp_custmer.json')
credit_df = spark.read.json('cdw_sapp_credit.json')
branch_df = spark.read.json('cdw_sapp_branch.json')

http://LAPTOP-2BOR1SQA:4041


<h2>Transform</h2>

In [63]:
customer_df.select('LAST_UPDATED').show()

+--------------------+
|        LAST_UPDATED|
+--------------------+
|2018-04-21T12:49:...|
|2018-04-21T12:49:...|
|2018-04-21T12:49:...|
|2018-04-21T12:49:...|
|2018-04-21T12:49:...|
|2018-04-21T12:49:...|
|2018-04-21T12:49:...|
|2018-04-21T12:49:...|
|2018-04-21T12:49:...|
|2018-04-21T12:49:...|
|2018-04-21T12:49:...|
|2018-04-21T12:49:...|
|2018-04-21T12:49:...|
|2018-04-21T12:49:...|
|2018-04-21T12:49:...|
|2018-04-21T12:49:...|
|2018-04-21T12:49:...|
|2018-04-21T12:49:...|
|2018-04-21T12:49:...|
|2018-04-21T12:49:...|
+--------------------+
only showing top 20 rows



In [40]:
from pyspark.sql.functions import concat_ws
#Transform customer data
customer_df = customer_df.withColumnRenamed("FIRST_NAME","First_Name")
customer_df = customer_df.withColumnRenamed("MIDDLE_NAME", "Middle_Name")
customer_df = customer_df.withColumnRenamed("LAST_NAME", "Last_Name")

customer_df = customer_df.withColumn('FULL_STREET_ADDRESS', concat_ws(' ', customer_df.STREET_NAME, customer_df.APT_NO))
customer_df = customer_df.drop('APT_NO', 'STREET_NAME')

customer_df.withColumn("CUST_PHONE", concat_ws('-', customer_df.CUST_PHONE[0:3], customer_df.CUST_PHONE[3:7])).show(2)

customer_df = customer_df.withColumn("CUST_ZIP",customer_df.CUST_ZIP.cast("int"))
customer_df = customer_df.drop_duplicates()
customer_df.show(2)
customer_df.printSchema()

+----------------+------------+-------------+-------------------+----------+----------+--------+----------+---------+--------------------+-----------+---------+--------------------+
|  CREDIT_CARD_NO|   CUST_CITY| CUST_COUNTRY|         CUST_EMAIL|CUST_PHONE|CUST_STATE|CUST_ZIP|First_Name|Last_Name|        LAST_UPDATED|Middle_Name|      SSN| FULL_STREET_ADDRESS|
+----------------+------------+-------------+-------------------+----------+----------+--------+----------+---------+--------------------+-----------+---------+--------------------+
|4210653310061055|     Natchez|United States|AHooper@example.com| 123-37818|        MS|   39120|      Alec|   Hooper|2018-04-21T12:49:...|         Wm|123456100|Main Street North...|
|4210653310102868|Wethersfield|United States|EHolman@example.com| 123-38933|        CT|   06109|      Etta|   Holman|2018-04-21T12:49:...|    Brendan|123453023|   Redwood Drive 829|
+----------------+------------+-------------+-------------------+----------+----------+---

In [41]:
#Transform branch data

from pyspark.sql.functions import unix_timestamp, to_timestamp
from pyspark.sql.types import TimestampType

#branch_df.select("LAST_UPDATED").show(truncate=False)
branch_df.na.fill(value=00000,subset=["BRANCH_ZIP"])
branch_df.withColumn("BRANCH_PHONE", concat_ws('-', branch_df.BRANCH_PHONE[0:3], branch_df.BRANCH_PHONE[3:7]))

branch_df = branch_df.withColumn("LAST_UPDATED",to_timestamp("LAST_UPDATED"))
branch_df.printSchema()
branch_df = branch_df.drop_duplicates()
branch_df.show(3)
branch_df.printSchema()


root
 |-- BRANCH_CITY: string (nullable = true)
 |-- BRANCH_CODE: long (nullable = true)
 |-- BRANCH_NAME: string (nullable = true)
 |-- BRANCH_PHONE: string (nullable = true)
 |-- BRANCH_STATE: string (nullable = true)
 |-- BRANCH_STREET: string (nullable = true)
 |-- BRANCH_ZIP: long (nullable = true)
 |-- LAST_UPDATED: timestamp (nullable = true)

+-----------+-----------+------------+------------+------------+-----------------+----------+-------------------+
|BRANCH_CITY|BRANCH_CODE| BRANCH_NAME|BRANCH_PHONE|BRANCH_STATE|    BRANCH_STREET|BRANCH_ZIP|       LAST_UPDATED|
+-----------+-----------+------------+------------+------------+-----------------+----------+-------------------+
|    Acworth|        197|Example Bank|  1234914637|          GA|  Briarwood Drive|     30101|2018-04-18 16:51:47|
|  Mundelein|         78|Example Bank|  1234164452|          IL|    Heather Court|     60060|2018-04-18 16:51:47|
|    Huntley|          2|Example Bank|  1234618993|          IL|Washington St

In [42]:
credit_df.show(1)
credit_df.printSchema()


+-----------+----------------+---------+---+-----+--------------+----------------+-----------------+----+
|BRANCH_CODE|  CREDIT_CARD_NO| CUST_SSN|DAY|MONTH|TRANSACTION_ID|TRANSACTION_TYPE|TRANSACTION_VALUE|YEAR|
+-----------+----------------+---------+---+-----+--------------+----------------+-----------------+----+
|        114|4210653349028689|123459988| 14|    2|             1|       Education|             78.9|2018|
+-----------+----------------+---------+---+-----+--------------+----------------+-----------------+----+
only showing top 1 row

root
 |-- BRANCH_CODE: long (nullable = true)
 |-- CREDIT_CARD_NO: string (nullable = true)
 |-- CUST_SSN: long (nullable = true)
 |-- DAY: long (nullable = true)
 |-- MONTH: long (nullable = true)
 |-- TRANSACTION_ID: long (nullable = true)
 |-- TRANSACTION_TYPE: string (nullable = true)
 |-- TRANSACTION_VALUE: double (nullable = true)
 |-- YEAR: long (nullable = true)



In [43]:
#Format credit data
#Concat YEAR, MONTH, DAY for YYYYMMDD format
from pyspark.sql.functions import date_format, col, to_date, udf, lpad, format_string
def day2d(d):
    print(d)
    return f'{d:02d}'


#day = credit_df.select(col('DAY'))

format_udf = udf(lambda x: f'{x:02s}')
credit_df = credit_df.withColumn("DAY",credit_df.DAY.cast("string"))
credit_df = credit_df.withColumn('DAY', lpad('DAY', 2, '0'))
credit_df = credit_df.withColumn("MONTH",credit_df.MONTH.cast("string"))
credit_df = credit_df.withColumn('MONTH', lpad('MONTH', 2, '0'))
#credit_df = credit_df.withColumn('MONTH', format_string('%02s', 'MONTH'))
#credit_df.select(col('DAY'), \
#format_string('%02s', credit_df.DAY).alias('Format')).show(3)

credit_df = credit_df.withColumn('TIMEID', concat_ws('', credit_df.YEAR, credit_df.MONTH, credit_df.DAY  ))
credit_df = credit_df.drop('DAY', 'MONTH', 'YEAR')

#credit_df = credit_df.drop_duplicates()
#credit_df.withColumn('TRANSACTION_ID', monotonically_increasing_id())
#credif_df = credit_df.select(date_format('TIMEID', 'yyyy MM dd').alias('TIMEID'))
#credit_df.select(col("TIMEID"),to_date(col("TIMEID"),"yyyyMMdd")).show()
#credit_df.selectExpr('to_date(TIMEID, "yyyyMMdd") as date').show()
#credit_df.show(3)


In [44]:

credit_df.printSchema()
#credit_df.filter(col('DAY').isNull()).show()
credit_df.show(3)


root
 |-- BRANCH_CODE: long (nullable = true)
 |-- CREDIT_CARD_NO: string (nullable = true)
 |-- CUST_SSN: long (nullable = true)
 |-- TRANSACTION_ID: long (nullable = true)
 |-- TRANSACTION_TYPE: string (nullable = true)
 |-- TRANSACTION_VALUE: double (nullable = true)
 |-- TIMEID: string (nullable = false)

+-----------+----------------+---------+--------------+----------------+-----------------+--------+
|BRANCH_CODE|  CREDIT_CARD_NO| CUST_SSN|TRANSACTION_ID|TRANSACTION_TYPE|TRANSACTION_VALUE|  TIMEID|
+-----------+----------------+---------+--------------+----------------+-----------------+--------+
|        114|4210653349028689|123459988|             1|       Education|             78.9|20180214|
|         35|4210653349028689|123459988|             2|   Entertainment|            14.24|20180320|
|        160|4210653349028689|123459988|             3|         Grocery|             56.7|20180708|
+-----------+----------------+---------+--------------+----------------+-----------------

<h2>Load</h2>

In [45]:
#Write dataframes to dbms
credit_df.write.format("jdbc") \
  .mode("overwrite") \
  .option("url", "jdbc:mysql://localhost:3306/creditcard_capstone") \
  .option("dbtable", "CDW_SAPP_CREDIT") \
  .option("driver", "com.mysql.jdbc.Driver") \
  .option("user", "root") \
  .option("password", "db") \
  .save()

branch_df.write.format("jdbc") \
  .mode("overwrite") \
  .option("url", "jdbc:mysql://localhost:3306/creditcard_capstone") \
  .option("dbtable", "CDW_SAPP_BRANCH") \
  .option("driver", "com.mysql.jdbc.Driver") \
  .option("user", "root") \
  .option("password", "db") \
  .save() 

customer_df.write.format("jdbc") \
  .mode("overwrite") \
  .option("url", "jdbc:mysql://localhost:3306/creditcard_capstone") \
  .option("dbtable", "CDW_SAPP_CUSTOMER ") \
  .option("driver", "com.mysql.jdbc.Driver") \
  .option("user", "root") \
  .option("password", "db") \
  .save() 


<h2>Req-2.1</h2>

In [46]:
#1)    Used to display the transactions made by customers living in a given zip code for a given month and year. Order by day in descending order.
credit_df.createOrReplaceTempView("credit_df")
customer_df.createOrReplaceTempView("customer_df")
branch_df.createOrReplaceTempView("branch_df")
"""customer_df.show(1)
credit_df.show(1)
branch_df.show(1)"""

def customer_by_zip_and_date(zip, month, year):
    string=year+month
    return spark.sql('WITH t AS \
( \
	SELECT EXTRACT(MONTH FROM to_date(TIMEID, "yyyyMMdd")) AS M, EXTRACT(YEAR FROM to_date(TIMEID, "yyyyMMdd")) AS Y \
	FROM credit_df \
) \
SELECT DISTINCT TRANSACTION_ID, t.M, t.Y \
FROM credit_df as c, customer_df as cu, t \
WHERE c.CUST_SSN = cu.SSN AND "%s"= cu.CUST_ZIP AND "%s"= t.M AND "%s"= t.Y' % (zip, month, year))

    #return spark.sql(f'SELECT TRANSACTION_ID FROM credit_df as c, customer_df as cu WHERE c.CUST_SSN = cu.SSN AND "%s"=cu.CUST_ZIP AND "s%"= CAST(c.TIMEID AS date)' % (zip, string))
    #return spark.sql(f'SELECT TRANSACTION_ID FROM credit_df as c, customer_df as cu WHERE c.CUST_SSN = cu.SSN AND "%s"=cu.CUST_ZIP AND "s%" LIKE c.TIMEID' % (zip, string))

#spark.sql('SELECT EXTRACT(MONTH FROM to_date(TIMEID, "yyyyMMdd")) from credit_df').show(3)

customer_by_zip_and_date('91740', '02', '2018').show()
#AS c, customer_df AS cu WHERE c.CUST_SSN = cu.SSN AND "s%"=cu.CUST_ZIP AND "s%" LIKE TIMEID" % (zip, string)


+--------------+---+----+
|TRANSACTION_ID|  M|   Y|
+--------------+---+----+
|         16995|  2|2018|
|          3949|  2|2018|
|         17008|  2|2018|
|         16982|  2|2018|
|         16989|  2|2018|
|         21086|  2|2018|
|          3980|  2|2018|
|          3988|  2|2018|
|         16973|  2|2018|
|         21077|  2|2018|
|         16977|  2|2018|
|         17011|  2|2018|
|          3996|  2|2018|
|          3962|  2|2018|
|         21084|  2|2018|
|         21079|  2|2018|
|          3970|  2|2018|
|          4007|  2|2018|
|         16992|  2|2018|
|          3990|  2|2018|
+--------------+---+----+
only showing top 20 rows



In [47]:
#2)    Used to display the number and total values of transactions for a given type.
def count_sum_by_type(t):
    return spark.sql(f'SELECT COUNT(TRANSACTION_ID), SUM(TRANSACTION_VALUE) FROM credit_df WHERE "%s"=TRANSACTION_TYPE' % (t))
string="Grocery"
count_sum_by_type(string).show()


+---------------------+----------------------+
|count(TRANSACTION_ID)|sum(TRANSACTION_VALUE)|
+---------------------+----------------------+
|                 6549|     337051.6299999992|
+---------------------+----------------------+



In [48]:
#3)    Used to display the number and total values of transactions for branches in a given state.
def count_sum_by_state(state):
    return spark.sql(f'SELECT COUNT(TRANSACTION_ID), SUM(TRANSACTION_VALUE) FROM credit_df as c, branch_df as b WHERE c.BRANCH_CODE=b.BRANCH_CODE AND "%s"=b.BRANCH_STATE' % (state))
count_sum_by_state("GA").show()


+---------------------+----------------------+
|count(TRANSACTION_ID)|sum(TRANSACTION_VALUE)|
+---------------------+----------------------+
|                 2023|             102716.99|
+---------------------+----------------------+



<h2> Req 2.2 </h2>

In [49]:
#1) Used to check the existing account details of a customer.
def customer(ssn):
    return spark.sql(f'SELECT * FROM customer_df WHERE "%s"=customer_df.SSN' % (ssn))
customer('123453723').show()

+----------------+---------+-------------+-----------------+----------+----------+--------+----------+---------+--------------------+-----------+---------+-------------------+
|  CREDIT_CARD_NO|CUST_CITY| CUST_COUNTRY|       CUST_EMAIL|CUST_PHONE|CUST_STATE|CUST_ZIP|First_Name|Last_Name|        LAST_UPDATED|Middle_Name|      SSN|FULL_STREET_ADDRESS|
+----------------+---------+-------------+-----------------+----------+----------+--------+----------+---------+--------------------+-----------+---------+-------------------+
|4210653321125592| Glendora|United States|MHays@example.com|   1242383|        CA|   91740|   Maritza|     Hays|2018-04-21T12:49:...|     Warner|123453723|       Route 32 100|
+----------------+---------+-------------+-----------------+----------+----------+--------+----------+---------+--------------------+-----------+---------+-------------------+



In [50]:
#2) Used to modify the existing account details of a customer.
#Input column and value to update
def update_customer(c, v, ssn):
    spark.sql(f'UPDATE customer_df SET "%s" = "%s" WHERE customer_df.SSN = "%s"' % (c, v) )

In [51]:
#3) Used to generate a monthly bill for a credit card number for a given month and year.
def generate_monthly_bill(cc, year, month):
    string = year+month

In [52]:
#4) Used to display the transactions made by a customer between two dates. Order by year, month, and day in descending order.
def transactions_between(date1, date2):
    return spark.sql('SELECT TRANSACTION_ID FROM credit_df as c WHERE to_date(c.TIMEID, "yyyyMMdd") > to_date("%s", "yyyyMMdd") AND to_date(c.TIMEID, "yyyyMMdd") < to_date("%s", "yyyyMMdd")' % (date1, date2))
transactions_between('20180213', '20180321' ).show()

+--------------+
|TRANSACTION_ID|
+--------------+
|             1|
|             2|
|             9|
|            23|
|            25|
|            41|
|            51|
|            59|
|            62|
|            68|
|            73|
|            76|
|            81|
|            82|
|            83|
|            99|
|           112|
|           134|
|           141|
|           149|
+--------------+
only showing top 20 rows



In [53]:
customer_df.show(1)
credit_df.show(3)
branch_df.show(1)

+----------------+---------+-------------+-----------------+----------+----------+--------+----------+---------+--------------------+-----------+---------+-------------------+
|  CREDIT_CARD_NO|CUST_CITY| CUST_COUNTRY|       CUST_EMAIL|CUST_PHONE|CUST_STATE|CUST_ZIP|First_Name|Last_Name|        LAST_UPDATED|Middle_Name|      SSN|FULL_STREET_ADDRESS|
+----------------+---------+-------------+-----------------+----------+----------+--------+----------+---------+--------------------+-----------+---------+-------------------+
|4210653321125592| Glendora|United States|MHays@example.com|   1242383|        CA|   91740|   Maritza|     Hays|2018-04-21T12:49:...|     Warner|123453723|       Route 32 100|
+----------------+---------+-------------+-----------------+----------+----------+--------+----------+---------+--------------------+-----------+---------+-------------------+
only showing top 1 row

+-----------+----------------+---------+--------------+----------------+-----------------+------

<h2> Req 3 </h2>

In [54]:
#Converting Spark to Pandas to use Plotly
cdf = credit_df.toPandas()
cdf.head()

bdf = branch_df.toPandas()
bdf.head()

cudf = customer_df.toPandas()

In [55]:
import plotly.express as px

In [56]:
#Find and plot which transaction type has a high rate of transactions.
grouped = cdf.groupby('TRANSACTION_TYPE')['TRANSACTION_ID'].count().reset_index()
grouped
#Do they sell testing products or is 'Test' a testing category? I guess we'll never know.
 
fig = px.bar(grouped, x='TRANSACTION_TYPE', y='TRANSACTION_ID', title='Transactions per Category', color="TRANSACTION_TYPE")
fig.show()

In [57]:
#Find and plot which state has a high number of customers.
grouped = cudf.groupby('CUST_STATE')['SSN'].count().reset_index()
grouped

fig = px.bar(grouped, x='CUST_STATE', y='SSN', color='CUST_STATE', title='Customers by State')
fig.show()

In [58]:
#Find and plot the sum of all transactions for each customer, and which customer has the highest transaction amount. hint(use CUST_SSN).
grouped = cdf.groupby('CUST_SSN')['TRANSACTION_ID'].count().reset_index()
grouped2 = cdf.groupby('CUST_SSN')['TRANSACTION_VALUE'].sum().reset_index()
hv_customer = grouped2.sort_values(by=['TRANSACTION_VALUE'], ascending=False).head(1)

fig = px.scatter(grouped2, x='CUST_SSN', y='TRANSACTION_VALUE', size="TRANSACTION_VALUE", color='TRANSACTION_VALUE') #What kind of company publishes SSNs in a chart, this one (•_•)  ( •_•)>⌐■-■  (⌐■_■)
fig.show()


In [59]:
#

rdd1 = spark.sparkContext.parallelize([1,2,3,4,5])
display(rdd1, rdd1.collect())

ParallelCollectionRDD[331] at readRDDFromFile at PythonRDD.scala:274

[1, 2, 3, 4, 5]

In [60]:
print(credit_df.schema.fields)

[StructField('BRANCH_CODE', LongType(), True), StructField('CREDIT_CARD_NO', StringType(), True), StructField('CUST_SSN', LongType(), True), StructField('TRANSACTION_ID', LongType(), True), StructField('TRANSACTION_TYPE', StringType(), True), StructField('TRANSACTION_VALUE', DoubleType(), True), StructField('TIMEID', StringType(), False)]


In [61]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, ArrayType
from pyspark.sql.functions import col,array_contains

spark = SparkSession.builder.appName('SparkApp').getOrCreate()
# ------create dummy data ----
arrayStructureData = [
        (("James","","Smith"),["Java","Scala","C++"],"OH","M"),
        (("Anna","Rose",""),["Spark","Java","C++"],"NY","F"),
        (("Julia","","Williams"),["CSharp","VB"],"OH","F"),
        (("Maria","Anne","Jones"),["CSharp","VB"],"NY","M"),
        (("Jen","Mary","Brown"),["CSharp","VB"],"NY","M"),
        (("Mike","Mary","Williams"),["Python","VB"],"OH","M")
        ]
        
#-----define the StructType and StructFields----
arrayStructureSchema = StructType([
        StructField('name', StructType([
             StructField('firstname', StringType(), True),
             StructField('middlename', StringType(), True),
             StructField('lastname', StringType(), True)
             ])),
         StructField('languages', ArrayType(StringType()), True),
         StructField('state', StringType(), True),
         StructField('gender', StringType(), True)
         ])

#---create the dataframe and add schema to the dataframe---
df = spark.createDataFrame(data = arrayStructureData, schema = arrayStructureSchema)
df.printSchema()
df.show(truncate=False)

#Equals
df.filter(df.state == "OH") \
    .show(truncate=False)

#Not equals
df.filter(~(df.state == "OH")) \
    .show(truncate=False)
df.filter(df.state != "OH") \
    .show(truncate=False)    
    
df.filter(col("state") == "OH") \
    .show(truncate=False)    
    
df.filter("gender  == 'M'") \
    .show(truncate=False)    

df.filter("gender  <> 'M'") \
    .show(truncate=False)    

#IS IN
li=["OH","CA","DE"]
df.filter(df.state.isin(li)).show()
#IS NOT IN
df.filter(~df.state.isin(li)).show()

df.filter( (df.state  == "OH") & (df.gender  == "M") ) \
    .show(truncate=False)        

df.filter(array_contains(df.languages,"Java")) \
    .show(truncate=False)        

df.filter(df.name.lastname == "Williams") \
    .show(truncate=False) 

df.filter(df.state.startswith("N")).show()
df.filter(df.state.endswith("H")).show()
df.filter(df.state.like("N%")).show()


root
 |-- name: struct (nullable = true)
 |    |-- firstname: string (nullable = true)
 |    |-- middlename: string (nullable = true)
 |    |-- lastname: string (nullable = true)
 |-- languages: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- state: string (nullable = true)
 |-- gender: string (nullable = true)

+----------------------+------------------+-----+------+
|name                  |languages         |state|gender|
+----------------------+------------------+-----+------+
|{James, , Smith}      |[Java, Scala, C++]|OH   |M     |
|{Anna, Rose, }        |[Spark, Java, C++]|NY   |F     |
|{Julia, , Williams}   |[CSharp, VB]      |OH   |F     |
|{Maria, Anne, Jones}  |[CSharp, VB]      |NY   |M     |
|{Jen, Mary, Brown}    |[CSharp, VB]      |NY   |M     |
|{Mike, Mary, Williams}|[Python, VB]      |OH   |M     |
+----------------------+------------------+-----+------+

+----------------------+------------------+-----+------+
|name                  |langu